In [2]:
from util import load_external_labels

# %time load_external_labels(session, Spouse, annotator_name='gold')

from snorkel.annotations import load_gold_labels
from snorkel import SnorkelSession
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

session = SnorkelSession()
train_cands = session.query(Spouse).filter(Spouse.split == 0).all()
dev_cands = session.query(Spouse).filter(Spouse.split == 1).all()
test_cands = session.query(Spouse).filter(Spouse.split == 2).all()


# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
# gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
gold_labels_dev = [L[0,0] for L in L_gold_dev]
gold_labels_test = [L[0,0] for L in L_gold_test]

from snorkel.learning.utils import MentionScorer

In [3]:
import numpy as np
unique, counts = np.unique(gold_labels_dev, return_counts=True)
print(dict(zip(unique, counts)))


unique, counts = np.unique(gold_labels_test, return_counts=True)
print(dict(zip(unique, counts)))

print(len(gold_labels_dev),len(gold_labels_test))

{0: 2625, 1: 189}
{0: 2484, 1: 218}
2814 2702


In [4]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]
    
output shape : [NoOfDataPoints,2,NoOfLFs]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for i,ci in enumerate(cands):
        L_S_ci=[]
        L=[]
        S=[]
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
    return L_S


In [5]:

# spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}

#drop 1
# spouses = {'spouse', 'ex-wife', 'ex-husband'}

#drop 2
spouses = {'spouse','husband', 'ex-husband'}




# family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
#               'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}

#drop 1
# family = {'sister', 'son', 'brother',
#               'grandfather', 'grandmother', 'aunt', 'cousin'}

#drop 2
family = {'father', 'mother', 'daughter',
              'grandfather', 'grandmother', 'uncle', 'cousin'}

family = family | {f + '-in-law' for f in family}
# other = {'boyfriend', 'girlfriend', 'boss', 'employee', 'secretary', 'co-worker'}

#drop 1
# other = {'boyfriend', 'girlfriend', 'boss', 'secretary'}

#drop 2
other = {'boyfriend', 'boss', 'employee', 'co-worker'}

In [6]:
import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

In [7]:
####### Discrete ##########




# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None    

def LF_husband_wife(c):
    return (1,1) if len(spouses.intersection(get_between_tokens(c))) > 0 else (0,0)

def LF_husband_wife_left_window(c):
    if len(spouses.intersection(get_left_tokens(c[0], window=2))) > 0:
        return (1,1)
    elif len(spouses.intersection(get_left_tokens(c[1], window=2))) > 0:
        return (1,1)
    else:
        return (0,0)
    
def LF_same_last_name(c):
    p1_last_name = last_name(c.person1.get_span())
    p2_last_name = last_name(c.person2.get_span())
    if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
        if c.person1.get_span() != c.person2.get_span():
            return (1,1)
    return (0,0)

def LF_no_spouse_in_sentence(c):
    return (-1,1) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

def LF_and_married(c):
    return (1,1) if 'and' in get_between_tokens(c) and 'married' in get_right_tokens(c) else (0,0)
    
def LF_familial_relationship(c):
    return (-1,1) if len(family.intersection(get_between_tokens(c))) > 0 else (0,0)

def LF_family_left_window(c):
    if len(family.intersection(get_left_tokens(c[0], window=2))) > 0:
        return (-1,1)
    elif len(family.intersection(get_left_tokens(c[1], window=2))) > 0:
        return (-1,1)
    else:
        return (0,0)

def LF_other_relationship(c):
    return (-1,1) if len(other.intersection(get_between_tokens(c))) > 0 else (0,0)


import bz2

# Function to remove special characters from text
def strip_special(s):
    return ''.join(c for c in s if ord(c) < 128)

# Read in known spouse pairs and save as set of tuples
with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'rb') as f:
    known_spouses = set(
        tuple(strip_special(x.decode('utf-8')).strip().split(',')) for x in f.readlines()
    )
# Last name pairs for known spouses
last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
def LF_distant_supervision(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

def LF_distant_supervision_last_names(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    p1n, p2n = last_name(p1), last_name(p2)
    return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,0)


LFs = [
    LF_distant_supervision, LF_distant_supervision_last_names, 
    LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
    LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
    LF_family_left_window, LF_other_relationship
]

In [8]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

dev_L_S = get_L_S_Tensor(dev_cands)

np.save("dev_L_S_discrete_drop2",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_cands)
np.save("train_L_S_discrete_drop2",np.array(train_L_S))


test_L_S = get_L_S_Tensor(test_cands)
np.save("test_L_S_discrete_drop2",np.array(test_L_S))



print("--- %s seconds ---" % (time.time() - start_time))



started at: 28-6-2018, 12:53:53
500data points labelled in 0.07170267105102539 mins
1000data points labelled in 0.12891732851664225 mins
1500data points labelled in 0.19803133408228557 mins
2000data points labelled in 0.23399905363718668 mins
2500data points labelled in 0.27207231521606445 mins
500data points labelled in 0.3719041387240092 mins
1000data points labelled in 0.41960506041844686 mins
1500data points labelled in 0.4680821259816488 mins
2000data points labelled in 0.5222727020581563 mins
2500data points labelled in 0.5665136655171712 mins
3000data points labelled in 0.6027114590009054 mins
3500data points labelled in 0.6385161916414896 mins
4000data points labelled in 0.6745089371999105 mins
4500data points labelled in 0.7082351406415304 mins
5000data points labelled in 0.7538342555363973 mins
5500data points labelled in 0.7945145765940348 mins
6000data points labelled in 0.8284995953241984 mins
6500data points labelled in 0.8740629355112711 mins
7000data points labelled in 

In [9]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    write_to_file(wordvec_unavailable)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [10]:
##### Continuous ################





# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None    

def LF_husband_wife(c):
    global LF_Threshold
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for sw in spouses:
        sc=max(sc,get_similarity(word_vectors,sw))
    return (1,sc)

def LF_husband_wife_left_window(c):
    global LF_Threshold
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for sw in spouses:
        sc_1=max(sc_1,get_similarity(word_vectors,sw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for sw in spouses:
        sc_2=max(sc_2,get_similarity(word_vectors,sw))
    return(1,max(sc_1,sc_2))
    
def LF_same_last_name(c):
    p1_last_name = last_name(c.person1.get_span())
    p2_last_name = last_name(c.person2.get_span())
    if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
        if c.person1.get_span() != c.person2.get_span():
            return (1,1)
    return (0,0)

def LF_no_spouse_in_sentence(c):
    return (-1,0.75) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

def LF_and_married(c):
    global LF_Threshold
    word_vectors = get_word_vectors(preprocess(get_right_tokens(c)))
    sc = get_similarity(word_vectors,'married')
    
    if 'and' in get_between_tokens(c):
        return (1,sc)
    else:
        return (0,0)

def LF_familial_relationship(c):
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for fw in family:
        sc=max(sc,get_similarity(word_vectors,fw))
        
    return (-1,sc) 

def LF_family_left_window(c):
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for fw in family:
        sc_1=max(sc_1,get_similarity(word_vectors,fw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for fw in family:
        sc_2=max(sc_2,get_similarity(word_vectors,fw))
        
    return (-1,max(sc_1,sc_2))

def LF_other_relationship(c):
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for ow in other:
        sc=max(sc,get_similarity(word_vectors,ow))
        
    return (-1,sc) 

# def LF_other_relationship_left_window(c):
#     sc = 0
#     word_vectors = get_word_vectors(preprocess(get_left_tokens(c)))
#     for ow in other:
#         sc=max(sc,get_similarity(word_vectors,ow))
#     return (-1,sc) 

import bz2

# Function to remove special characters from text
def strip_special(s):
    s = s.decode("utf-8") 
    return ''.join(c for c in s if (ord(c) < 128))


# Read in known spouse pairs and save as set of tuples
with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'r') as f:
    known_spouses = set(
        tuple(strip_special(x).strip().split(',')) for x in f.readlines()
    )
# Last name pairs for known spouses
last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
def LF_distant_supervision(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

def LF_distant_supervision_last_names(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    p1n, p2n = last_name(p1), last_name(p2)
    return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,1)

import numpy as np

# def LF_Three_Lists_Left_Window(c):
#     global softmax_Threshold
#     c1,s1 = LF_husband_wife_left_window(c)
#     c2,s2 = LF_family_left_window(c)
#     c3,s3 = LF_other_relationship_left_window(c)
#     sc = np.array([s1,s2,s3])
#     c = [c1,c2,c3]
#     sharp_param = 1.5
#     prob_sc = np.exp(sc * sharp_param - np.max(sc))
#     prob_sc = prob_sc / np.sum(prob_sc)
#     #print 'Left:',s1,s2,s3,prob_sc
    
#     if s1==s2 or s3==s1:
#         return (0,0)
#     return c[np.argmax(prob_sc)],1

# def LF_Three_Lists_Between_Words(c):
#     global softmax_Threshold
#     c1,s1 = LF_husband_wife(c)
#     c2,s2 = LF_familial_relationship(c)
#     c3,s3 = LF_other_relationship(c)
#     sc = np.array([s1,s2,s3])
#     c = [c1,c2,c3]
#     sharp_param = 1.5
    
#     prob_sc = np.exp(sc * sharp_param - np.max(sc))
#     prob_sc = prob_sc / np.sum(prob_sc)
#     #print 'BW:',s1,s2,s3,prob_sc
#     if s1==s2 or s3==s1:
#         return (0,0)
#     return c[np.argmax(prob_sc)],1
    
LFs = [
    LF_distant_supervision, LF_distant_supervision_last_names, 
    LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
    LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
    LF_family_left_window, LF_other_relationship
]

In [11]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

dev_L_S = get_L_S_Tensor(dev_cands)

np.save("dev_L_S_smooth_drop2",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_cands)
np.save("train_L_S_smooth_drop2",np.array(train_L_S))


test_L_S = get_L_S_Tensor(test_cands)
np.save("test_L_S_smooth_drop2",np.array(test_L_S))



print("--- %s seconds ---" % (time.time() - start_time))



started at: 28-6-2018, 12:58:49
500data points labelled in 0.47844405968983966 mins
1000data points labelled in 0.8854199926058451 mins
1500data points labelled in 1.2853091915448507 mins
2000data points labelled in 1.7044175108273825 mins
2500data points labelled in 2.1445502479871115 mins
500data points labelled in 2.813088798522949 mins
1000data points labelled in 3.237121891975403 mins
1500data points labelled in 3.743137756983439 mins
2000data points labelled in 4.1531915465990705 mins
2500data points labelled in 4.609000166257222 mins
3000data points labelled in 5.142787102858225 mins
3500data points labelled in 5.5737285852432255 mins
4000data points labelled in 5.992953185240427 mins
4500data points labelled in 6.461323873202006 mins
5000data points labelled in 6.871107006072998 mins
5500data points labelled in 7.321115958690643 mins
6000data points labelled in 7.7784007549285885 mins
6500data points labelled in 8.195359218120576 mins
7000data points labelled in 8.7790183424949

In [13]:
import pandas as pd
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import average_precision_score

In [29]:
LF_l = [
    1,1,1,1,1,-1,1,-1,-1,-1
]

dev_L_S = np.load("dev_L_S_discrete_drop2.npy")
test_L_S = np.load("test_L_S_discrete_drop2.npy")
train_L_S = np.load("train_L_S_discrete_drop2.npy")
print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)

(2702, 2, 10) (2814, 2, 10) (22276, 2, 10)


In [15]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32
seed = 12

In [19]:
import scipy.sparse as sp
import _pickle as pkl


L_train = np.load("train_L_S_discrete_drop2.npy")
L_train = L_train[:,0,:].astype(int)
print(np.array(L_train).shape)
L_train = sp.csr_matrix(L_train)


from snorkel.learning import GenerativeModel
import time
import datetime

gen_model = GenerativeModel()

start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

gen_model.train(L_train, epochs = 100)
# gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)


print("trained in ",str(datetime.timedelta(seconds=time.time() - start_time)))

(22276, 10)
started at: 28-6-2018, 14:31:23
Inferred cardinality: 2
trained in  0:01:18.605451


In [20]:

L_dev = np.load("dev_L_S_discrete_drop2.npy")
L_dev = L_dev[:,0,:].astype(int)
print(np.array(L_dev).shape)
L_dev = sp.csr_matrix(L_dev)

marginals = gen_model.marginals(L_dev)
marginals = np.array(marginals)
print(marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(gold_labels_dev),GenLabels,average="binary"))


(2814, 10)
(2814,)
(2814,)
(0.3357142857142857, 0.24867724867724866, 0.2857142857142857, None)


In [21]:

L_test = np.load("test_L_S_discrete_drop2.npy")
L_test = L_test[:,0,:].astype(int)
print(np.array(L_test).shape)
L_test = sp.csr_matrix(L_test)

marginals = gen_model.marginals(L_test)
marginals = np.array(marginals)
print(marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(gold_labels_test),GenLabels,average="binary"))


(2702, 10)
(2702,)
(2702,)
(0.3670212765957447, 0.3165137614678899, 0.3399014778325123, None)


In [26]:
NoOfLFs= len(LF_l)
NoOfClasses = 2
print(len(LF_l))

10


In [24]:

def train(lr,ep,th,af,batch_size=32,LF_acc=None,LF_rec=None,pcl=np.array([-1,1],dtype=np.float64),norm=True,\
          smooth=True,penalty=0,p3k=3,alp=1,Gamma=1.0,debug=True):
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## pcl : all possible class labels  = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    ## alp : alpha parameter (to set a max value for alpha)
    ## norm : use normalization or not
    ## Gamma : penalty tuning parameter
    
    BATCH_SIZE = batch_size
    tf.reset_default_graph()
    

    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(len(dev_L_S))
        test_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)
        test_init_op = iterator.make_initializer(test_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)
        
        

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    
        g = tf.convert_to_tensor(Gamma, dtype=tf.float64)
        
        if(penalty in [4,5,7,8,9,10,11]):
            LF_a = tf.convert_to_tensor(LF_acc, dtype=tf.float64)
        
        if(penalty in [6,7,11,12]):
            LF_r = tf.convert_to_tensor(LF_rec, dtype=tf.float64)
        
        if(debug):
            print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
        if(debug):
            print("s",s)
            print("l",l)
#         print(s.graph)
        if(smooth):
            s_ = tf.maximum(tf.subtract(s,tf.minimum(alphas,alp)), 0)
            if(debug):
                print("s_",s_)

       
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
            if(debug):
                print("out",out)
            return out

        if(smooth):
            pout = tf.map_fn(lambda c: l*c*s_ ,pcl,name="pout")
        else:
            pout = tf.map_fn(lambda c: l*c ,pcl,name="pout")

        if(debug):
            print("pout",pout)    

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                           name="t_pout")
    
        if(debug):
            print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        if(debug):
            print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
            if(debug):
                print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
            if(debug):
                print("intsy",out1)
            return out1
                

        if(smooth):
            #smooth normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="zy")
        else:
            #discrete normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="zy")
            
        ### for precision and recall t_pout
        def p_t_pout(j):
            Lj = tf.map_fn(lambda li : tf.gather(li,j),l)
            if(debug):
                print("sft Lj",Lj)
            kj = tf.gather(k,j)
            if(debug):
                print("sft kj",kj)
            indices = tf.where(tf.equal(Lj,kj))
            if(debug):
                print("sft indices",indices)
            li_lij_eq_kj = tf.gather(l,tf.squeeze(indices,1))
            if(smooth):
                si_lij_eq_kj = tf.gather(s_,tf.squeeze(indices,1))
            if(debug):
                print("sft l_ij_eq_kj",li_lij_eq_kj)
            if(smooth):
                prec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="prec_zy"))
            else:
                prec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="prec_zy"))
            if(debug):
                print("prec_z",prec_z)
            if(smooth):
                prec_t_pout = (tf.matmul(li_lij_eq_kj*si_lij_eq_kj*kj, thetas,transpose_b=True))/prec_z
            else:
                prec_t_pout = (tf.matmul(li_lij_eq_kj*kj, thetas,transpose_b=True))/prec_z
            if(debug):
                print("prec_t_pout",prec_t_pout)
            return prec_t_pout
        
        def r_t_pout(j):
            kj = tf.gather(k,j)
            if(debug):
                print("r_t_pout kj",kj)

            if(smooth):
                rec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="prec_zy"))
            else:
                rec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="prec_zy"))
            if(debug):
                print("prec_z",prec_z)
            if(smooth):
                rec_t_pout = (tf.matmul(l*s_*kj, thetas,transpose_b=True))/rec_z
            else:
                rec_t_pout = (tf.matmul(l*kj, thetas,transpose_b=True))/rec_z
            if(debug):
                print("prec_t_pout",prec_t_pout)
            return rec_t_pout
            
           
        def softplus_p(j):
            aj = tf.gather(LF_a,j)
            if(debug):
                print("sft aj",aj)
            f_p =  tf.reduce_sum(aj - p_t_pout(j))
            if(debug):
                print("f_p",f_p)
            sft_p = tf.nn.softplus(f_p,name="sft_p")
            if(debug):
                print("sft_p",sft_p)
            return sft_p
        
       
        def softplus_r(j):
            rj = tf.gather(LF_r,j)
            if(debug):
                print("sft aj",rj)
            
            f_r =  rj - tf.reduce_sum(r_t_pout(j))
            if(debug):
                print("f_r",f_r)
            sft_r = tf.nn.softplus(f_r,name="sft_r")
            if(debug):
                print("sft_r",sft_r)
            return sft_r
        
              
#         logsft = tf.map_fn(lambda j: tf.log(softplus(j)),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)
#         sft  =  tf.map_fn(lambda j: softplus(j),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)
        
# 
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        if(debug):
            print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        if(debug):
            print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
        if(debug):
            print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))
        
        if(not norm):
            print("unnormlized loss")
            loss = tf.negative(tf.reduce_sum(lsp  ))
        elif(penalty == 1):
            print("penalty1")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +(g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 2):
            print("penalty2")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -(g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 3):
            print("penalty3")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +(g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k))))
        elif(penalty == 4):
            print("precision penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 5):
            print("precision log(softplus) penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: tf.log(softplus_p(j)),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 6):
            print("recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 7):
            print("precision and recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 8):
            print("precision and sign 1 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 9):
            print("precision and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 10):
            print("precision and sign 3 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k))))
        elif(penalty == 11):
            print("precision and recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 12):
            print("recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))

        else:
            print("normalized loss")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  ))
       
        if(debug):
            print("loss",loss)
        tf.summary.scalar('loss', loss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        if(debug):
            print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(loss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,loss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)
                print("dev set")
                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
                print(a)
                print(t)
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_dev,pl))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()
                print("test set")
                sess.run(test_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_test,pl))
                print(precision_recall_fscore_support(np.array(gold_labels_test),np.array(pl),average="binary"))
                print()
                
#             # Initialize an iterator over the validation dataset.
#             sess.run(dev_init_op)
#             a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
#             print(a)
#             print(t)

#             unique, counts = np.unique(pl, return_counts=True)
#             print(dict(zip(unique, counts)))

#             print("acc",accuracy_score(true_labels,pl))

# #             predictAndPrint(pl)
#             print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [30]:
train(0.1/len(train_L_S),5,batch_size = 1, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=False)

k Tensor("Const:0", shape=(10,), dtype=float64)
s Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
normalized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 178572.12323578287
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04]
[[1.01874622 0.71625187 0.91686

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [31]:
LF_l = [
    1,1,1,1,1,-1,1,-1,-1,-1
]
import numpy as np
dev_L_S = np.load("dev_L_S_smooth_drop2.npy")
test_L_S = np.load("test_L_S_smooth_drop2.npy")
train_L_S = np.load("train_L_S_smooth_drop2.npy")

# dev_L_S = np.load("dev_L_S_discrete.npy")
# train_L_S = np.load("train_L_S_discrete.npy")
# true_labels = gold_labels_test
print(len(gold_labels_dev),len(gold_labels_test))
print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)

2814 2702
(2702, 2, 10) (2814, 2, 10) (22276, 2, 10)


In [27]:
for i in np.linspace(0.6,1,5):
    print("alpha-init:",i)
    train(0.1/len(train_L_S),5,batch_size = 1, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=0,debug=False)

alpha-init: 0.6
normalized loss
0 loss 151736.46601589455
dev set
[0.67943488 0.64962526 0.67902256 0.6733065  0.65662258 0.52223885
 0.64508197 0.51289417 0.51114081 0.51392111]
[[1.04218139 0.76958075 0.94568324 0.94182309 1.05947376 1.12021438
  1.10207399 1.09851353 1.20839933 1.12987434]]
{0: 2807, 1: 7}
acc 0.9331911869225302
(0.5714285714285714, 0.021164021164021163, 0.04081632653061224, None)

test set
{0: 2691, 1: 11}
acc 0.9196891191709845
(0.5454545454545454, 0.027522935779816515, 0.05240174672489083, None)

1 loss 145448.2787580717
dev set
[0.64583942 0.63130564 0.7369439  0.72416167 0.66108734 0.44631264
 0.66133641 0.41529171 0.4130385  0.4164121 ]
[[1.0249567  0.76590069 0.89679007 0.89842829 1.04473895 1.19796216
  1.07914381 1.17881825 1.28863522 1.21187624]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

test set
{0: 2701, 1: 1}
acc 0.9189489267209474
(0.0, 0.0, 0.0, None)



/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


2 loss 137175.77142888107
dev set
[0.56427868 0.58321142 0.78313334 0.76401845 0.64242932 0.37428607
 0.66143567 0.31572418 0.31352986 0.3168538 ]
[[1.06413495 0.78173677 0.85712539 0.86322851 1.04308612 1.27328185
  1.08694296 1.26649548 1.37745467 1.30208621]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

test set
{0: 2702}
acc 0.9193190229459659
(0.0, 0.0, 0.0, None)

3 loss 127938.12427523505
dev set
[0.47119437 0.51530263 0.82138813 0.79720247 0.60534665 0.30850919
 0.64964689 0.21605314 0.2139103  0.2171926 ]
[[1.12836862 0.81378582 0.82426517 0.83431791 1.05189839 1.34490481
  1.11484228 1.35759598 1.46965585 1.39555104]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

test set
{0: 2702}
acc 0.9193190229459659
(0.0, 0.0, 0.0, None)

4 loss 118604.26466141483
dev set
[0.37369633 0.43378777 0.85307768 0.82493791 0.55110937 0.25141526
 0.63289453 0.11657885 0.11448985 0.11773103]
[[1.2078074  0.86175645 0.79715397 0.81044091 1.07189296 1.41181919
  1.15544135 1.44

0 loss 154393.3036334753
dev set
[1.1255746  0.99159964 1.12538509 0.99312141 1.12557231 1.12559795
 1.1255952  1.12558711 1.12562103 1.12560606]
[[1.24461897 0.82277695 1.13596607 1.00904842 1.23341181 1.16815575
  1.29646712 1.15797076 1.27354325 1.19163685]]
{0: 2712, 1: 102}
acc 0.925728500355366
(0.4019607843137255, 0.21693121693121692, 0.28178694158075607, None)

test set
{0: 2549, 1: 153}
acc 0.9130273871206513
(0.4444444444444444, 0.3119266055045872, 0.3665768194070081, None)

1 loss 154240.45881500412
dev set
[1.23195274 0.98671774 1.23182891 0.97468494 1.2319561  1.23205668
 1.23194827 1.23205232 1.23202199 1.23205085]
[[1.35319678 0.82901062 1.2445802  1.02925583 1.34199224 1.27678344
  1.40503316 1.26660188 1.38214321 1.30025768]]
{0: 2712, 1: 102}
acc 0.925728500355366
(0.4019607843137255, 0.21693121693121692, 0.28178694158075607, None)

test set
{0: 2549, 1: 153}
acc 0.9130273871206513
(0.4444444444444444, 0.3119266055045872, 0.3665768194070081, None)

2 loss 153644.21917

In [32]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.85)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.85*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=2,debug=False)

[18934.6 18934.6 18934.6 18934.6 18934.6 18934.6 18934.6 18934.6 18934.6
 18934.6]
batch-size: 128 alpha-init: 0.6
penalty2
0 loss 154092.68942425994
dev set
[0.60195673 0.59891914 0.60087865 0.60078679 0.6018443  0.59963373
 0.60247941 0.5995315  0.60068342 0.59986702]
[[1.11676472 0.81344419 1.00877584 0.99983877 1.10560172 1.04211838
  1.1684198  1.03198389 1.1471093  1.06549265]]
{0: 2708, 1: 106}
acc 0.9164889836531628
(0.2830188679245283, 0.15873015873015872, 0.2033898305084746, None)

test set
{0: 2605, 1: 97}
acc 0.9056254626202813
(0.30927835051546393, 0.13761467889908258, 0.19047619047619047, None)

1 loss 154055.31048494493
dev set
[0.60273257 0.59969182 0.60166225 0.60156801 0.6026196  0.59885167
 0.60325236 0.59874644 0.59989842 0.59908177]
[[1.11598955 0.81267605 1.00799818 0.99906322 1.10482967 1.04290162
  1.1676435  1.0327561  1.14787109 1.06626012]]
{0: 2715, 1: 99}
acc 0.9175550817341862
(0.2828282828282828, 0.14814814814814814, 0.19444444444444445, None)

test set
{

2 loss 154384.046308082
dev set
[0.90335918 0.89917783 0.89853834 0.89847486 0.90161883 0.90273903
 0.90399298 0.8986416  0.89997811 0.90198092]
[[1.11554037 0.81410762 1.01125844 1.00228828 1.10680011 1.03902414
  1.16694309 1.03304501 1.14797754 1.06381835]]
{0: 2469, 1: 345}
acc 0.8869936034115139
(0.3130434782608696, 0.5714285714285714, 0.40449438202247195, None)

test set
{0: 2296, 1: 406}
acc 0.8778682457438934
(0.3620689655172414, 0.6743119266055045, 0.4711538461538461, None)

3 loss 154383.6142707817
dev set
[0.90407695 0.89951456 0.89802827 0.89797627 0.90180066 0.90350261
 0.90474802 0.89808787 0.89948504 0.90240738]
[[1.1148897  0.81408439 1.01181807 1.00283337 1.10694967 1.03826924
  1.1662065  1.03365567 1.14852138 1.06353321]]
{0: 2469, 1: 345}
acc 0.8869936034115139
(0.3130434782608696, 0.5714285714285714, 0.40449438202247195, None)

test set
{0: 2296, 1: 406}
acc 0.8778682457438934
(0.3620689655172414, 0.6743119266055045, 0.4711538461538461, None)

4 loss 154383.1840444

4 loss 154347.51857943484
dev set
[0.70312243 0.7000738  0.70204269 0.70195202 0.70300491 0.69844718
 0.70363647 0.6983616  0.69953254 0.69870473]
[[1.11560124 0.81230936 1.00761452 0.99868628 1.1044574  1.04330871
  1.16725624 1.03311397 1.14816053 1.06612209]]
{0: 2658, 1: 156}
acc 0.9150675195451314
(0.33974358974358976, 0.2804232804232804, 0.3072463768115942, None)

test set
{0: 2525, 1: 177}
acc 0.9093264248704663
(0.423728813559322, 0.3440366972477064, 0.37974683544303794, None)

batch-size: 256 alpha-init: 0.8
penalty2
0 loss 154398.45751003962
dev set
[0.80156589 0.79851676 0.80048058 0.80038789 0.80144483 0.80001854
 0.80208775 0.79995426 0.80115023 0.80100164]
[[1.11715727 0.81387203 1.00921998 1.00029136 1.10603188 1.04173334
  1.1688123  1.03157128 1.14666613 1.06442946]]
{0: 2585, 1: 229}
acc 0.8990760483297797
(0.2925764192139738, 0.3544973544973545, 0.3205741626794258, None)

test set
{0: 2440, 1: 262}
acc 0.8934122871946706
(0.366412213740458, 0.44036697247706424, 0.400

3 loss 154078.39872592737
dev set
[0.60196034 0.59892611 0.60088301 0.60079259 0.60184916 0.599627
 0.60247909 0.59952468 0.60067636 0.59986011]
[[1.11676007 0.81343196 1.00877267 0.99983786 1.10559325 1.04212528
  1.16841998 1.03199302 1.14711626 1.06550768]]
{0: 2708, 1: 106}
acc 0.9164889836531628
(0.2830188679245283, 0.15873015873015872, 0.2033898305084746, None)

test set
{0: 2606, 1: 96}
acc 0.9059955588452998
(0.3125, 0.13761467889908258, 0.1910828025477707, None)

4 loss 154068.9480547072
dev set
[0.60215594 0.59912147 0.60108032 0.60098948 0.60204489 0.59942973
 0.60267506 0.5993269  0.60047865 0.59966232]
[[1.11656449 0.81323676 1.00857606 0.9996417  1.10539761 1.04232279
  1.16822426 1.03218977 1.14731149 1.06570451]]
{0: 2708, 1: 106}
acc 0.9164889836531628
(0.2830188679245283, 0.15873015873015872, 0.2033898305084746, None)

test set
{0: 2607, 1: 95}
acc 0.9063656550703183
(0.3157894736842105, 0.13761467889908258, 0.19169329073482427, None)

batch-size: 512 alpha-init: 0.7


penalty2
0 loss 154405.46569398642
dev set
[1.00137947 0.99797639 1.00032022 1.000036   1.00126843 1.00062655
 1.00189418 1.00052849 1.00166616 1.00085578]
[[1.11768804 0.81438369 1.00956152 1.00062431 1.10650897 1.04136608
  1.1693828  1.03122316 1.14650066 1.06478942]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

1 loss 154405.46563686457
dev set
[1.00160806 0.99779889 1.00064107 1.00035683 1.00149996 1.00089523
 1.00211373 1.00080981 1.00188902 1.00110501]
[[1.11789563 0.81456968 1.0095938  1.00062663 1.10671107 1.04146895
  1.16959972 1.03130097 1.14671564 1.06494417]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

2 l

3 loss 154397.7485533031
dev set
[0.80156825 0.79853133 0.80048244 0.80039251 0.80145517 0.80001971
 0.8020858  0.79993881 0.80111735 0.80103191]
[[1.11715258 0.813835   1.00918844 1.00025312 1.10599611 1.04173232
  1.16881279 1.03158373 1.14669403 1.06436341]]
{0: 2585, 1: 229}
acc 0.8990760483297797
(0.2925764192139738, 0.3544973544973545, 0.3205741626794258, None)

test set
{0: 2440, 1: 262}
acc 0.8934122871946706
(0.366412213740458, 0.44036697247706424, 0.4000000000000001, None)

4 loss 154397.28742282343
dev set
[0.80166636 0.79862857 0.80057889 0.80048867 0.80155281 0.79992121
 0.80218399 0.79984606 0.80103151 0.80112759]
[[1.11705462 0.81374016 1.00909596 1.00016043 1.10590081 1.041831
  1.1687147  1.0316773  1.14678316 1.064274  ]]
{0: 2585, 1: 229}
acc 0.8990760483297797
(0.2925764192139738, 0.3544973544973545, 0.3205741626794258, None)

test set
{0: 2440, 1: 262}
acc 0.8934122871946706
(0.366412213740458, 0.44036697247706424, 0.4000000000000001, None)

batch-size: 1024 alpha-

penalty2
0 loss 154376.3757444092
dev set
[0.70122476 0.69819139 0.70014499 0.70005569 0.70111309 0.70036405
 0.70174139 0.70026268 0.70141388 0.70059778]
[[1.11749534 0.81416538 1.00951021 1.00057486 1.10632867 1.04138789
  1.16915664 1.03125724 1.14638386 1.06477035]]
{0: 2655, 1: 159}
acc 0.9147121535181236
(0.33962264150943394, 0.2857142857142857, 0.31034482758620685, None)

test set
{0: 2519, 1: 183}
acc 0.9071058475203553
(0.4098360655737705, 0.3440366972477064, 0.37406483790523687, None)

1 loss 154375.56085722346
dev set
[0.70127398 0.69824063 0.70019419 0.7001049  0.70116238 0.70031484
 0.70179054 0.70021352 0.7013649  0.70054861]
[[1.11744614 0.81411621 1.00946096 1.00052567 1.1062794  1.04143709
  1.1691074  1.03130626 1.14643231 1.06481733]]
{0: 2655, 1: 159}
acc 0.9147121535181236
(0.33962264150943394, 0.2857142857142857, 0.31034482758620685, None)

test set
{0: 2519, 1: 183}
acc 0.9071058475203553
(0.4098360655737705, 0.3440366972477064, 0.37406483790523687, None)

2 loss

3 loss 154405.4656680052
dev set
[1.00137941 0.99794856 1.00032661 1.00015666 1.00126838 1.00062699
 1.00189407 1.00052931 1.00166607 1.00085589]
[[1.11772968 0.81441004 1.00956727 1.00062457 1.10655741 1.0414151
  1.16940234 1.03125665 1.14652796 1.06485592]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

4 loss 154405.46565238794
dev set
[1.00143408 0.99789929 1.00039997 1.00023544 1.00132356 1.00068891
 1.00194721 1.00059376 1.00171977 1.00091418]
[[1.11778389 0.81446097 1.00957725 1.00062587 1.1066114  1.04145307
  1.16945629 1.03128639 1.14658216 1.06490499]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)



In [33]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.35*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=6,debug=False)

[7796.6 7796.6 7796.6 7796.6 7796.6 7796.6 7796.6 7796.6 7796.6 7796.6]
batch-size: 128 alpha-init: 0.6
recall penalty
0 loss 13798149.168170523
dev set
[0.60195673 0.59891912 0.60087865 0.60078678 0.60184429 0.59963373
 0.6024794  0.59953152 0.60068344 0.59986703]
[[1.11676473 0.81344424 1.00877585 0.99983878 1.10560175 1.04211838
  1.16841982 1.03198384 1.14710921 1.06549255]]
{0: 2708, 1: 106}
acc 0.9164889836531628
(0.2830188679245283, 0.15873015873015872, 0.2033898305084746, None)

test set
{0: 2605, 1: 97}
acc 0.9056254626202813
(0.30927835051546393, 0.13761467889908258, 0.19047619047619047, None)

1 loss 13798111.885884486
dev set
[0.60273256 0.59969178 0.60166225 0.60156801 0.60261958 0.59885166
 0.60325235 0.59874646 0.59989845 0.5990818 ]
[[1.11598957 0.81267615 1.0079982  0.99906324 1.10482974 1.04290163
  1.16764354 1.03275601 1.14787091 1.06625993]]
{0: 2715, 1: 99}
acc 0.9175550817341862
(0.2828282828282828, 0.14814814814814814, 0.19444444444444445, None)

test set
{0: 26

2 loss 13798434.070041906
dev set
[0.9033582  0.89916658 0.89852452 0.89846175 0.90160065 0.90273896
 0.90399277 0.89865786 0.89999523 0.90199817]
[[1.1155426  0.81411867 1.0112638  1.00229332 1.10681403 1.03902419
  1.16694363 1.03304089 1.14797307 1.06380233]]
{0: 2469, 1: 345}
acc 0.8869936034115139
(0.3130434782608696, 0.5714285714285714, 0.40449438202247195, None)

test set
{0: 2296, 1: 406}
acc 0.8778682457438934
(0.3620689655172414, 0.6743119266055045, 0.4711538461538461, None)

3 loss 13798433.639663342
dev set
[0.90407566 0.89949956 0.89800974 0.89795889 0.90177642 0.90350249
 0.90474776 0.89810952 0.89950773 0.90243031]
[[1.11489269 0.81409911 1.01182536 1.00284012 1.10696827 1.03826933
  1.16620721 1.03365005 1.14851533 1.06351205]]
{0: 2469, 1: 345}
acc 0.8869936034115139
(0.3130434782608696, 0.5714285714285714, 0.40449438202247195, None)

test set
{0: 2296, 1: 406}
acc 0.8778682457438934
(0.3620689655172414, 0.6743119266055045, 0.4711538461538461, None)

4 loss 13798433.21

4 loss 7015359.192627073
dev set
[0.70312242 0.7000737  0.70204265 0.70195197 0.70300484 0.69844716
 0.70363646 0.6983618  0.69953296 0.69870511]
[[1.11560127 0.81230963 1.00761462 0.9986864  1.10445764 1.04330873
  1.16725627 1.03311355 1.14815952 1.06611426]]
{0: 2658, 1: 156}
acc 0.9150675195451314
(0.33974358974358976, 0.2804232804232804, 0.3072463768115942, None)

test set
{0: 2525, 1: 177}
acc 0.9093264248704663
(0.423728813559322, 0.3440366972477064, 0.37974683544303794, None)

batch-size: 256 alpha-init: 0.8
recall penalty
0 loss 7015407.858507658
dev set
[0.80156587 0.79851663 0.80048041 0.80038771 0.8014447  0.80001854
 0.80208771 0.79995533 0.80115223 0.80100217]
[[1.11715729 0.81387243 1.00922148 1.00029269 1.10603245 1.04173333
  1.16881231 1.03157079 1.14666521 1.06442833]]
{0: 2585, 1: 229}
acc 0.8990760483297797
(0.2925764192139738, 0.3544973544973545, 0.3205741626794258, None)

test set
{0: 2440, 1: 262}
acc 0.8934122871946706
(0.366412213740458, 0.44036697247706424, 0

1 loss 3584607.6615848737
dev set
[0.60156851 0.59853461 0.60048913 0.60039925 0.60145705 0.60002058
 0.60208697 0.59991903 0.60107058 0.60025433]
[[1.11715182 0.81382304 1.00916561 1.00023007 1.10598511 1.0417314
  1.16881196 1.03160071 1.14672694 1.06511546]]
{0: 2706, 1: 108}
acc 0.9157782515991472
(0.2777777777777778, 0.15873015873015872, 0.20202020202020202, None)

test set
{0: 2601, 1: 101}
acc 0.9041450777202072
(0.297029702970297, 0.13761467889908258, 0.18808777429467086, None)

2 loss 3584598.3078152416
dev set
[0.60176451 0.59873047 0.60068592 0.60059577 0.6016532  0.59982396
 0.60228304 0.59972208 0.60087372 0.60005745]
[[1.11695587 0.81362741 1.00896919 1.00003399 1.10578909 1.04192813
  1.1686159  1.03179665 1.14692139 1.06531131]]
{0: 2707, 1: 107}
acc 0.916133617626155
(0.2803738317757009, 0.15873015873015872, 0.20270270270270271, None)

test set
{0: 2603, 1: 99}
acc 0.9048852701702442
(0.30303030303030304, 0.13761467889908258, 0.18927444794952683, None)

3 loss 3584588.

3 loss 3584888.6973123704
dev set
[0.90194487 0.89866963 0.89940895 0.89932365 0.90139636 0.9011981
 0.90247351 0.89961093 0.9008075  0.90125615]
[[1.11679515 0.8141581  1.01027127 1.00133211 1.10666719 1.04055498
  1.16842897 1.03194042 1.14703871 1.06425968]]
{0: 2469, 1: 345}
acc 0.8869936034115139
(0.3130434782608696, 0.5714285714285714, 0.40449438202247195, None)

test set
{0: 2296, 1: 406}
acc 0.8778682457438934
(0.3620689655172414, 0.6743119266055045, 0.4711538461538461, None)

4 loss 3584888.565133973
dev set
[0.90213697 0.89880161 0.89923982 0.89915703 0.90148203 0.90139332
 0.90266792 0.89943789 0.90064604 0.90140835]
[[1.11660822 0.81414701 1.01044699 1.00150593 1.10674195 1.04036048
  1.16823614 1.03212239 1.14721239 1.06414364]]
{0: 2469, 1: 345}
acc 0.8869936034115139
(0.3130434782608696, 0.5714285714285714, 0.40449438202247195, None)

test set
{0: 2296, 1: 406}
acc 0.8778682457438934
(0.3620689655172414, 0.6743119266055045, 0.4711538461538461, None)

batch-size: 512 alph

0 loss 1869652.5239701087
dev set
[0.80127382 0.79823932 0.80019344 0.80010425 0.80116154 0.80031461
 0.80179092 0.80021637 0.80137422 0.80074394]
[[1.11744648 0.81411988 1.00946513 1.00053003 1.1062831  1.04143731
  1.16910731 1.03130369 1.14642706 1.06463199]]
{0: 2585, 1: 229}
acc 0.8990760483297797
(0.2925764192139738, 0.3544973544973545, 0.3205741626794258, None)

test set
{0: 2439, 1: 263}
acc 0.8930421909696521
(0.3650190114068441, 0.44036697247706424, 0.3991683991683992, None)

1 loss 1869652.0672687523
dev set
[0.80137193 0.79833665 0.80028972 0.80020032 0.80125955 0.80021641
 0.80188925 0.80012408 0.80128909 0.80084018]
[[1.11734857 0.81402496 1.00937327 1.00043817 1.10618728 1.04153549
  1.16900912 1.0313969  1.14651588 1.06454224]]
{0: 2585, 1: 229}
acc 0.8990760483297797
(0.2925764192139738, 0.3544973544973545, 0.3205741626794258, None)

test set
{0: 2439, 1: 263}
acc 0.8930421909696521
(0.3650190114068441, 0.44036697247706424, 0.3991683991683992, None)

2 loss 1869651.611

3 loss 1011735.4246987357
dev set
[0.60137249 0.59833918 0.60029268 0.60020324 0.60126095 0.60021647
 0.60188931 0.60011512 0.60126643 0.60045025]
[[1.11734762 0.81401758 1.00936235 1.000427   1.10618073 1.04153547
  1.16900887 1.03140499 1.14653168 1.06492048]]
{0: 2706, 1: 108}
acc 0.9157782515991472
(0.2777777777777778, 0.15873015873015872, 0.20202020202020202, None)

test set
{0: 2599, 1: 103}
acc 0.9034048852701703
(0.2912621359223301, 0.13761467889908258, 0.18691588785046728, None)

4 loss 1011733.0830159871
dev set
[0.60142173 0.59838842 0.60034198 0.60025251 0.60131022 0.60016721
 0.60193856 0.60006582 0.60121715 0.60040095]
[[1.11729839 0.81396835 1.00931304 1.00037772 1.10613146 1.04158473
  1.16895964 1.03145427 1.14658085 1.0649698 ]]
{0: 2706, 1: 108}
acc 0.9157782515991472
(0.2777777777777778, 0.15873015873015872, 0.20202020202020202, None)

test set
{0: 2599, 1: 103}
acc 0.9034048852701703
(0.2912621359223301, 0.13761467889908258, 0.18691588785046728, None)

batch-size: 

0 loss 1012031.4654412807
dev set
[1.00122492 0.99809366 1.00014453 1.00001149 1.00111327 1.00046302
 1.0017415  1.00036175 1.0015128  1.00069679]
[[1.11758478 0.81426297 1.00955983 1.00062431 1.10641555 1.04134694
  1.16925195 1.03121199 1.14637892 1.06474444]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

1 loss 1012031.4654209154
dev set
[1.00127506 0.99804568 1.00019867 1.00003549 1.00116349 1.00051445
 1.00179138 1.00041369 1.00156278 1.00074755]
[[1.11763009 0.8143112  1.00956051 1.00062431 1.10645938 1.04136256
  1.16930029 1.03122081 1.14642641 1.06477528]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

2 loss 10120

In [34]:
#majority vote #84 # Regex
# train_L_S_df.mode(axis=1)[0].tolist()
import math
# test_L_S_df.mode(axis=1)[0].tolist()
test_L_S_df = pd.DataFrame((np.array(test_L_S)[:,0,:]))
# test_L_S_df = test_L_S_df.astype(int)
test_L_S_df = test_L_S_df.replace(0, np.NaN)

predicted_labels=test_L_S_df.mode(axis=1)[0].tolist()

predicted_labels = [ int(x) if not math.isnan(x) else 1 for x in predicted_labels ]
predicted_labels = [ 0 if x == -1 else 1 for x in predicted_labels ]
# print(predicted_labels)
# Results = predictAndPrint(predicted_labels)
print("acc",accuracy_score(gold_labels_test,predicted_labels))
print(precision_recall_fscore_support(np.array(gold_labels_test),np.array(predicted_labels),average="binary"))
           

acc 0.6902294596595114
(0.17386722866174922, 0.7568807339449541, 0.2827763496143959, None)


In [35]:
#majority vote #84 # Regex
# train_L_S_df.mode(axis=1)[0].tolist()
import math
# test_L_S_df.mode(axis=1)[0].tolist()
test_L_S_df = pd.DataFrame((np.array(test_L_S)[:,0,:]))
# test_L_S_df = test_L_S_df.astype(int)
test_L_S_df = test_L_S_df.replace(0, np.NaN)

predicted_labels=test_L_S_df.mode(axis=1)[0].tolist()

predicted_labels = [ int(x) if not math.isnan(x) else -1 for x in predicted_labels ]
predicted_labels = [ 0 if x == -1 else 1 for x in predicted_labels ]
# print(predicted_labels)
# Results = predictAndPrint(predicted_labels)
print("acc",accuracy_score(gold_labels_test,predicted_labels))
print(precision_recall_fscore_support(np.array(gold_labels_test),np.array(predicted_labels),average="binary"))
           

acc 0.8915618060695781
(0.39669421487603307, 0.6605504587155964, 0.49569707401032703, None)
